In [55]:
# all important imports
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup 
from selenium import webdriver
import random
import time
import json

In [2]:
# opens web browser
dr = webdriver.Chrome()

In [3]:
# gets some pick up lines so you dont have to bother
pick_up_phrases = 'http://pun.me/pages/cheesy-pick-up-lines.php'
page = uReq(pick_up_phrases)
bs = BeautifulSoup(page.read(),"html.parser")

pick_up_lines = list()

lines = bs.find_all('li', href = False)
for line in lines:
    if '<a' in str(line):
        continue
    else:
        pick_up_lines.append(line.text)

In [4]:
pick_up_phrases = 'https://thoughtcatalog.com/kendra-syrdal/2016/05/101-online-dating-openers-that-are-way-better-than-just-saying-hi/'
page = uReq(pick_up_phrases)
bs = BeautifulSoup(page.read(),"html.parser")

items = bs.find('div',{'class':'entry'})

lines = items.find_all('p')

for line in lines:
    pick_up_lines.append(line.text)

In [5]:
# opens the browser and lets you log in unless you login dont start next process
main_link = 'https://www.okcupid.com'
# to enter your credentials
my_url_Page = main_link+'/doubletake'
dr.get(my_url_Page)

In [70]:
people = list()

In [73]:
# check the person. If match rate is more than 80% then sends of random pick up line
for a in range (1,2):
    # person, pick up line
    person = dict()

    # to make less predictable for the webpage
    rand_timeout = random.randint(5,10)

    my_url_Page = main_link+'/doubletake'
    dr.get(my_url_Page)
    bs = BeautifulSoup(dr.page_source,"html.parser")

    match_prc = bs.find('span',{'class':'cardsummary-match-pct'}).text
    match_number = int(match_prc.replace('%',''))


    # gathers details about person
    details = bs.find_all('div',{'class':'matchprofile-details-text'})

    person_details = list()

    for detail in details:
        person_details.append(detail.text.strip())


    # gathers all about part
    all_about = bs.find_all('div',{'class':'qmessays-essay'})

    about_person = list()

    for about in all_about:
        person_desc = about.find('p').text.strip()
        person_desc_name = about.find('h2').text.strip()
        about_dict = dict()
        about_dict[person_desc_name] = person_desc
        about_person.append(about_dict)

    person['name'] = bs.find('div',{'class':'cardsummary-realname'}).text.strip()
    person['age'] = bs.find('div',{'class':'cardsummary-age'}).text.strip()
    person['location'] = bs.find('div',{'class':'cardsummary-location'}).text.strip()
    person['match_percent'] = match_number
    person['pick_up_line'] = pick_up_line
    person['about'] = about_person
    person['details'] = person_details

    # if the match rate is more than 90
    if match_number > 90:
        # just in case something goes wrong or response is not delivered in time
        match_profile = bs.find('div',{'class':'cardsummary-item cardsummary-profile-link'})
        new_link = main_link + match_profile.find('a',href = True)['href']
        dr.get(new_link)
        button_like = dr.find_element_by_id('like-button')
        button_like.click()
        
        try:
            time.sleep(rand_timeout)

            # to make it more customizable we can add name in from.
            pick_up_line = random.choice(pick_up_lines).strip()    

            time.sleep(rand_timeout)

            text = dr.find_element_by_class_name('messenger-composer')
            text.send_keys("Hey ",person['name'],'. ',pick_up_line)
            time.sleep(1)
            button_send = dr.find_element_by_class_name('messenger-toolbar-send')
            button_send.click()

            person['matched'] = True

            time.sleep(rand_timeout)

        except:
            pass
    else:
        button_reject = dr.find_element_by_class_name('cardactions-action')
        button_reject.click()
        person['matched'] = False
        time.sleep(rand_timeout)

    people.append(person)


In [74]:
with open('people.json', 'w') as outfile:
    json.dump(people, outfile)